In [123]:

path='/home/greg/freefem/balltube/35_2_2_0.3_0.1'

with open(path+'/bdata.txt', 'r') as f:
    bda = f.readline().split(',')
    #bda=map(float, bda[:-1])
with open(path+'/mnums.txt', 'r') as f:
    nums = f.read().splitlines()
    ends0, evals, modes=map(int,nums[:3])
    lengths0= map(float,nums[-2:])
    ends=int(bda[0])
lengths=map(float,bda[1+4*ends:-1])
with open(path+'/ppmcalcsize.txt', 'r') as f:
    bda=f.read().splitlines()
    modes1 ,pp, ends1=int(bda[0]),float(bda[1]),int(bda[2])

    
if (ends0==ends and ends==ends1):
    print(ends,'ends ok')
else:
    print('ends mismatch, something is wrong')
    
if (lengths==lengths0):
    print(lengths,'lengths ok')
else:
    print('lengths mismatch, something is wrong')
    
if (modes==modes1):
    print(modes,'modes ok')
else:
    print('modes mismatch, something is wrong')
    
print(evals)

rmodes=modes
revals=evals


with open(path+'/ppmndfcs.txt', 'r') as f:
   ndm04 = map(float,f.read().splitlines())
   ndm04= column_matrix(RDF,modes*ends,ndm04)
   
rndm04=ndm04
   
with open(path+'/meval.txt', 'r') as f:
   mus = map(float,f.read().splitlines())

rmus = mus   
   
with open(path+'/mevec.txt', 'r') as f:
   mevec = map(float,f.read().splitlines())
   mevec =  matrix(CDF,evals,mevec)
   
rmevec=mevec
rmevecT=rmevec.T

zeros=[0 for i in range(ndm04.nrows())]
rzeros=[0 for i in range(ndm04.nrows())]
zeromodes=[[0] for i in range(ends)]
smdim=len(flatten(zeromodes))
zeropoz=[]
dzeropoz=[]
kernelpoz=[]
dkernelpoz=[]    
mmlengths=[j for j in lengths for i in range(1,rmodes)]
mmevs=[i for j in lengths for i in range(0,rmodes-1)]
DZEROV=zero_vector(2*ends*rmodes)
ZEROV=zero_vector(ends*rmodes)

def ndm(l):
    components(x)=(pp-l)/(l*pp-l*x-pp*x+x^2)
    co=map(components,mus)
    return ndm04-(mevec.T)*diagonal_matrix(CDF,co)*mevec


def dndm(l):
   components(x)=-1/(x-l)^2
   co=map(components,mus)
   return (mevec.T)*diagonal_matrix(CDF,co)*mevec    
    
def ndm0(l):
   components(x)=1/(x-l)
   co=map(components,mus)
   return (mevec.T)*diagonal_matrix(CDF,co)*mevec
   
def mm(l):
   f(x,y)=1/(I*sqrt(l-(x*pi/y)^2))
   return diagonal_matrix(CDF,flatten([map(f(x,i),srange(modes)) for i in lengths]))
   
def dmm(l):
   f(x,y)=-(I*sqrt(l-(x*pi/y)^2))
   return diagonal_matrix(CDF,flatten([map(f(x,i),srange(modes)) for i in lengths]))

#BIG weakness here need to adapt to differetn lengths
def diagdiff(l):
   f=lambda y:I*sqrt(l-(y*pi/2)^2)
   if imaginary(l)<0:
       return -diagonal_matrix(CDF,map(f,flatten(zeromodes)))
   else:
       return diagonal_matrix(CDF,map(f,flatten(zeromodes)))
   

  

def rndm(l):
    components=lambda x:(pp-l)/(l*pp-l*x-pp*x+x^2)
    co=diagonal_matrix(CDF,map(components,rmus))
    return rndm04-(rmevecT)*co*rmevec


   

def drndm(l):
    components=lambda x:1/((l-x)^2)
    co=diagonal_matrix(CDF,map(components,rmus))
    return (rmevecT)*co*rmevec


   
  
def rmm(l):
   f=lambda x,y: 1/(I*sqrt(l-(x*pi/y)^2))
   return diagonal_matrix(CDF,map(f,mmevs,mmlengths))



def drmm(l):
   f=lambda x,y:-I*sqrt(l-(x*pi/y)^2)
   return diagonal_matrix(CDF,map(f,mmevs,mmlengths))
   


class change(object):    
    @staticmethod
    def numbers(rm,rev):
        int(rev)
        int(rm)
        global rmodes
        global revals
        global rndm04
        global rmevec
        global rmevecT
        global rmus
        global rzeros
        global mmlengths
        global mmevs
        global DZEROV
        global ZEROV
        rmus = mus[:rev]
        rmodes=rm
        revals=rev
        modetrimmer=flatten([map(lambda m : m+i*modes,range(rm)) for i in range(ends)])
        rndm04=ndm04.matrix_from_rows_and_columns(modetrimmer,modetrimmer)
        rmevec=mevec.submatrix(row=0,nrows=rev)
        rmevec=rmevec.matrix_from_columns(modetrimmer)
        rmevecT=rmevec.T
        rzeros=[0 for i in range(rndm04.nrows())]
        change.modes(zeromodes)
        mmlengths=[j for j in lengths for i in range(0,rmodes)]
        mmevs=[i for j in lengths for i in range(0,rmodes)]
        DZEROV=zero_vector(2*ends*rmodes)
        ZEROV=zero_vector(ends*rmodes)
        
    @staticmethod
    def modes(zm):
        global r0mevec
        global r0ndm04
        global zeropoz
        global dzeropoz
        global zeromodes
        global kernelpoz
        global dkernelpoz
        zeromodes=zm
        global smdim
        zeropoz=[]
        
        test0=[0 for i in range(rmodes*ends)]
        test0col=[0 for i in range(revals)]
        test0=[0 for i in range(rmodes*ends)]
        test0col=[0 for i in range(revals)]
        r0mevec=deepcopy(rmevec)
        r0ndm04=deepcopy(rndm04)
        for i in range (0,ends):
            for j in zm[i]: 
                r0mevec.set_column((i*rmodes)+j,test0col)
                r0ndm04.set_row((i*rmodes)+j,test0)
                zeropoz.append((i*rmodes)+j)
        kernelpoz=[rmodes*ends-i-1 for i in range(len(zeropoz)) ]
        smdim=len(flatten(zeromodes))
        kernelpoz.reverse()
        dkernelpoz=map(lambda x:x+rmodes*ends,kernelpoz)
        dzeropoz=map(lambda x:x+rmodes*ends,zeropoz)
        

   

def xxxn(l):
    killer=map(lambda m :m*modes,range(ends))
    return (ndm(l)-mm(l)).delete_columns(killer).delete_rows(killer)
    

def rxxxn(l):
    killer=map(lambda m :m*rmodes,range(ends))
    return (rndm(l)-rmm(l)).delete_columns(killer).delete_rows(killer)
    

   

def sm(l):
    srndm=rndm(l)
    srmm=rmm(l)
    dd=diagdiff(l)
    rxxxs=srndm-srmm
    for i in zeropoz: rxxxs.set_row(i,ZEROV) 
    
    W=(rxxxs.SVD()[2]).matrix_from_columns(kernelpoz)
    
    basis1=W.matrix_from_rows(zeropoz)
    basis2=(srndm*W).matrix_from_rows(zeropoz).inverse()
    psm=basis1*basis2
    return (psm-dd).inverse()*(psm+dd)
    


 

def dsm(l):
    
    
    
    
    zb=matrix(rmodes*ends)
    
    irndm=rndm(l)
    idrndm=drndm(l)
    idintsystem = irndm.augment(zb).stack((idrndm).augment(irndm))
    
    irmm=rmm(l)
    idrmm=drmm(l)
    idrmmsys=(-idrmm.augment(zb).stack((0.5*irmm).augment(idrmm))).inverse()
    
    svdoutput =idintsystem-idrmmsys
    for i in zeropoz: svdoutput.set_row(i,DZEROV) 
    
    dd=diagdiff(l)
    ddtwo=-0.5*dd.inverse()
    W=(svdoutput.SVD()[2]).matrix_from_columns(dkernelpoz)
    A=[W.matrix_from_rows(zeropoz),W.matrix_from_rows(dzeropoz)]
    WB=idintsystem*W
    B=[WB.matrix_from_rows(zeropoz).inverse(),WB.matrix_from_rows(dzeropoz).inverse()]
    P=[A[0]*B[0],A[1]*B[1]]
    S=(P[0]-dd).inverse()*(dd+P[0])
    D=(P[1]-dd).inverse()*(ddtwo*S+ddtwo)
    return [S,D]

@parallel
def jacobi(l):
    idsm=dsm(l)
    return (idsm[0].inverse()*idsm[1]).trace()

def simpsonator(a,b,density):
    simpnum=max(int(abs(b-a))*density*2+1,3)
    h=(b-a)/simpnum
    iptb=[a+i*h for i in xrange(0,simpnum+1)]
    #print(iptb)
    jacs=jacobi(iptb)
    sumtab=[jacs.next()[1]]
    for i in xrange(0,(len(iptb)-1)/2):
        #print(i)
        sumtab.append(2*jacs.next()[1])
        sumtab.append(4*jacs.next()[1])
    sumtab.append(jacs.next()[1])
    return sumtab

def simsum(a,b,density):
    simpnum=max(int(abs(b-a))*density*2+1,3)
    h=(b-a)/simpnum
    iptb=[a+i*h for i in xrange(0,simpnum+1)]
    print(simpnum)
    jacs=jacobi(iptb)  
    sumtab=jacs.next()[1]
    for i in xrange(0,(len(iptb)-1)/2):
        sumtab += 2*jacs.next()[1]
        sumtab += 4*jacs.next()[1]
    sumtab += jacs.next()[1]
    return h*sumtab/3

def npsimpsonator(a,b,density):
    simpnum=max(int(abs(b-a))*density*2+1,3)
    h=(b-a)/simpnum
    iptb=[a+i*h for i in xrange(0,simpnum+1)]
    print(simpnum)
    jacs=[]
    for i in iptb:
        jacs.append(jacobi(i))  
    return jacs

def mrsimsum(a,b,density):
    simpnum=max(int(abs(b-a))*density*2+1,3)
    h=(b-a)/simpnum
    iptb=[a+i*h for i in xrange(0,simpnum+1)]
    map(jacobi,iptb)
    return h/3
        
def divsimsum(a,b,density):
    simpnum=max(int(abs(b-a))*density*2+1,3)
    h=(b-a)/simpnum
    iptb=[a+i*h for i in xrange(0,simpnum+1)]
    map(jacobi,iptb)
    return h/3

(2, 'ends ok')
([2.0, 2.0], 'lengths ok')
(20, 'modes ok')
1000


In [137]:
change.modes([[0],[0]])
change.numbers(10,600)
#mrsimsum(2+I,3+I,10)

#pretty_print(altrmm(2.3))
#pretty_print(rmm(2.3))
#timeit('simsum(2+I,3+I,10)',number=1,repeat=1)
timeit('dsm(2+I)')

5 loops, best of 3: 2.79 s per loop


In [136]:
pretty_print(dsm(2.3+I))

[
[0.027623819356739986 - 0.14846367640351454*I  0.4952332942368341 + 0.024560875805182823*I]  [ 0.08826619418107298 - 0.01991535108766277*I  0.044782167605486756 + 0.3166635844455686*I]
[  0.4952332942332953 + 0.02456087580455708*I 0.027623828573046835 - 0.14846366851079984*I], [ 0.044782167604939346 + 0.3166635844439314*I 0.08826619949365186 - 0.019915343194006764*I]
]

In [26]:
change.modes([[0],[0]])
change.numbers(10,600)
a=jacobi([0.01*i for i in xrange(10,20)])
#timeit('sum(x[1] for x in a)')
print(a.next()[1])
print(a.next())
print(a.next())

6.88338275268e-15 + 6.57519927635*I
(((0.110000000000000,), {}), 2.6645352591003757e-14 + 6.26838175066635*I)
(((0.120000000000000,), {}), -1.0880185641326534e-14 + 6.0007323654797435*I)


In [126]:
S = RecursivelyEnumeratedSet( [[]], lambda l: [l+[0], l+[1]] if len(l) < 16 else [], structure='forest', enumeration='depth')
S

An enumerated set with a forest structure